In [1]:
import pandas as pd
import requests
import time
import json
import os
from kafka import KafkaProducer

START_DATE = "2021-08-01"
END_DATE = "2025-01-30"
GEOGRAPHY_FILE = "../../data/source/thailand_geography.csv"


In [2]:
KAFKA_BROKER = 'localhost:9092' 
KAFKA_TOPIC = 'air_data'

producer = KafkaProducer(
    bootstrap_servers=[KAFKA_BROKER],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)
print(f"✅ Connected to Kafka Broker: {KAFKA_BROKER}")

✅ Connected to Kafka Broker: localhost:9092


In [3]:
# ---------------------------------------------------------
# PREPARE LOCATIONS
# ---------------------------------------------------------
target_locations = []
try:
    df_geo = pd.read_csv(GEOGRAPHY_FILE)
    bangkok_districts = df_geo[df_geo['province'].str.contains("Bangkok|กรุงเทพ", case=False, na=False)]

    
    for index, row in bangkok_districts.iterrows():
        target_locations.append({
            "name": row['district'], 
            "lat": row['latitude'],
            "lon": row['longitude']
        })

    print(f"🎯 Target: {len(target_locations)} districts")
except FileNotFoundError:
    print(f"❌ Error: หาไฟล์ {GEOGRAPHY_FILE} ไม่เจอ")

# ---------------------------------------------------------
# SCRAPING & SENDING LOOP
# ---------------------------------------------------------
print(f"🚀 Starting Data Collection -> Kafka Topic: {KAFKA_TOPIC}")
total_sent = 0

for location in target_locations:
    dist_name = location['name']
    
    url = "https://air-quality-api.open-meteo.com/v1/air-quality"
    
    params = {
        "latitude": location['lat'],
        "longitude": location['lon'],
        "start_date": START_DATE,
        "end_date": END_DATE,
        "hourly": "pm2_5",
        "timezone": "Asia/Bangkok"
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # --- PROCESS: Hourly -> Daily Average ---
        hourly_data = data.get('hourly', {})
        times = hourly_data.get('time', [])
        values = hourly_data.get('pm2_5', [])
        
        temp_df = pd.DataFrame({
            'time': pd.to_datetime(times),
            'pm25': values
        })
        
        # Resample เป็นรายวัน (หาค่าเฉลี่ย)
        temp_df.set_index('time', inplace=True)
        daily_df = temp_df.resample('D').mean().reset_index()
        
        # --- SEND TO KAFKA ---
        # วนลูปส่งทีละวันของเขตรนั้นๆ
        for _, row in daily_df.iterrows():
            
            # เช็คว่าค่า PM2.5 ไม่ใช่ NaN (API บางทีข้อมูลหาย)
            pm25_val = row['pm25']
            if pd.isna(pm25_val):
                continue

            payload = {
                "district": dist_name,
                "date": row['time'].strftime('%Y-%m-%d'), # แปลงเป็น string วันที่
                "avg_pm25_ug_m3": float(pm25_val),        # แปลงเป็น float
                "latitude": location['lat'],
                "longitude": location['lon']
            }
            
            # ส่งเข้า Kafka
            producer.send(KAFKA_TOPIC, value=payload)
            total_sent += 1

        print(f"✓ Sent data for {dist_name} , records: {len(daily_df)} , total sent: {total_sent}")
        
    except Exception as e:
        print(f"x Failed {dist_name}: {e}")
        
    # Be polite to API
    time.sleep(0.1)

# ---------------------------------------------------------
# 4. FINISH
# ---------------------------------------------------------
producer.flush() # ดันข้อมูลที่ค้างท่อออกให้หมด
producer.close()

print("------------------------------------------------")
print(f"✅ Completed! Sent total {total_sent} records to Kafka.")

🎯 Target: 169 districts
🚀 Starting Data Collection -> Kafka Topic: air_data
✓ Sent data for พระนคร , records: 1279 , total sent: 911
✓ Sent data for พระนคร , records: 1279 , total sent: 1822
✓ Sent data for พระนคร , records: 1279 , total sent: 2733
✓ Sent data for พระนคร , records: 1279 , total sent: 3644
✓ Sent data for พระนคร , records: 1279 , total sent: 4555
✓ Sent data for พระนคร , records: 1279 , total sent: 5466
✓ Sent data for พระนคร , records: 1279 , total sent: 6377
✓ Sent data for พระนคร , records: 1279 , total sent: 7288
✓ Sent data for พระนคร , records: 1279 , total sent: 8199
✓ Sent data for พระนคร , records: 1279 , total sent: 9110
✓ Sent data for พระนคร , records: 1279 , total sent: 10021
✓ Sent data for พระนคร , records: 1279 , total sent: 10932
✓ Sent data for ดุสิต , records: 1279 , total sent: 11843
✓ Sent data for ดุสิต , records: 1279 , total sent: 12754
✓ Sent data for ดุสิต , records: 1279 , total sent: 13665
✓ Sent data for ดุสิต , records: 1279 , total sent: 1